In [3]:
import pandas as pd

Données sur le PIB


Nom du dataframe: df_GDP

Cette copie comporte des data frames similaires à ceux du premier document. Seule différence: les indices comportent les dates (apparemment c'est mieux pour une analyse en séries temporelles)

In [ ]:
import pandas as pd

# Extraction des données relatives au GDP (Données Eurostat)
'''
Informations générales
Online data code:namq_10_gdp
Unit of measure: Current prices-million euro
'''

# Charger les données
df_GDP = pd.read_csv("Données_extraites/GDP_trimestriel_eurostat.csv", encoding='utf-8')
#print(df_GDP.head())

# Choix des colonnes rélévantes
df_GDP_selected = df_GDP[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_GDP_selected.head())

# Obtention des données de panel
df_GDP_pivot = df_GDP_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure un préfixe "PIB_" suivi du nom du pays
df_GDP_pivot.columns = [f'PIB_{col}' for col in df_GDP_pivot.columns]

# Vérification
df_GDP=df_GDP_pivot
df_GDP



,PIB_Albania,PIB_Austria,PIB_Belgium,PIB_Bosnia and Herzegovina,PIB_Bulgaria,PIB_Croatia,PIB_Cyprus,PIB_Czechia,PIB_Denmark,PIB_Estonia,...,PIB_Portugal,PIB_Romania,PIB_Serbia,PIB_Slovakia,PIB_Slovenia,PIB_Spain,PIB_Sweden,PIB_Switzerland,PIB_Türkiye,PIB_United Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,NaN,43186.6,53331.5,NaN,3175.8,3998.7,1730.6,10278.9,34078.4,647.2,...,21844.5,6370.1,6454.4,3608.0,3698.7,109201.7,49061.2,64546.7,NaN,257589.8
1995-Q2,NaN,46786.8,56119.3,NaN,3162.8,4481.5,1928.9,11487.7,35871.0,766.7,...,22939.5,6573.1,7824.6,4193.6,4215.6,118545.9,49347.7,67655.1,NaN,248722.5
1995-Q3,NaN,46335.5,53741.7,NaN,3796.9,4599.3,1984.4,12011.0,34625.0,764.9,...,22831.1,8410.6,8671.4,3935.9,4192.8,115932.3,47946.2,67566.8,NaN,255889.4
1995-Q4,NaN,47362.4,57115.3,NaN,4339.3,4443.3,1951.0,12547.9,36803.0,810.2,...,23393.0,7093.1,6061.2,3773.0,4243.8,126388.4,58220.1,70342.0,NaN,263308.8
1996-Q1,NaN,44641.2,54240.0,NaN,3757.2,4155.4,1806.6,11817.6,35666.0,810.8,...,22578.3,5736.8,3836.3,4047.6,3796.9,120868.5,55464.8,67670.1,NaN,262343.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q4,6084.2,122280.3,158085.7,6603.6,27190.9,19504.2,8018.3,80924.4,96868.6,10167.4,...,69392.1,97353.1,20553.9,32337.7,16574.7,396068.0,143722.2,213788.9,278047.3,NaN
2024-Q1,5428.8,117304.1,147812.8,6224.7,22171.0,18370.7,7847.3,73617.4,91782.6,8928.0,...,66207.1,70386.4,18778.1,29968.5,15715.2,377866.0,137093.7,212946.8,263622.5,NaN
2024-Q2,6513.1,120746.2,154840.7,6505.9,24615.2,21259.2,8266.2,81119.0,98319.4,9770.0,...,70879.5,84568.9,20490.7,32467.5,16806.2,400518.0,142633.5,211841.3,284656.9,NaN


Dans ce qui suit: nous allons essayer de concatener trois datasets:
- 2 comportant des données trimestrielles (1998-2008 et 2008-2024)
- 1 comporatant des données annuelles (1995-1998)

Nous avons gardé la même source (Eurostat) afin d'avoir des estimations homogènes


In [ ]:
import pandas as pd

# Extraction des données Working Hours per year (Données Eurostat: 1995-1997)
'''
Informations générales
Lien: https://doi.org/10.2908/LFSA_EWHANA
Code: lfsa_ewhana
'''

# Charger les données
df_working_hours_eurostat_an = pd.read_csv("Données_extraites/Working_hours_eurostat_1995-1998.csv", encoding='utf-8')
#print(df_working_hours_eurostat_an.head())

# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_an = df_working_hours_eurostat_an[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_working_hours_selected_eurostat_an.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_an = df_working_hours_selected_eurostat_an.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure un préfixe "WorkingHours_" suivi du nom du pays
df_working_hours_pivot_eurostat_an.columns = [f'Working_Hours_{col}' for col in df_working_hours_pivot_eurostat_an.columns]

# Vérifier que les dates sont bien en index et les pays sont renommés
#print(df_working_hours_pivot_eurostat_an.head())
df_working_hours_pivot_eurostat_an


,Working_Hours_Austria,Working_Hours_Belgium,Working_Hours_Czechia,Working_Hours_Denmark,Working_Hours_Estonia,Working_Hours_Finland,Working_Hours_France,Working_Hours_Germany,Working_Hours_Greece,Working_Hours_Hungary,...,Working_Hours_Netherlands,Working_Hours_Norway,Working_Hours_Poland,Working_Hours_Portugal,Working_Hours_Romania,Working_Hours_Slovenia,Working_Hours_Spain,Working_Hours_Sweden,Working_Hours_Switzerland,Working_Hours_United Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995,40.0,37.8,NaN,34.3,NaN,36.9,39.7,38.7,42.3,NaN,...,30.5,34.4,NaN,41.6,NaN,NaN,38.3,34.0,NaN,36.9
1996,39.8,37.8,NaN,34.0,NaN,37.6,39.6,38.7,41.9,41.2,...,32.3,34.9,NaN,40.8,NaN,42.2,38.0,34.4,37.0,36.7
1997,40.2,37.9,43.4,34.4,41.3,39.4,39.2,38.6,41.8,42.0,...,32.3,35.7,42.2,40.0,40.3,40.5,38.3,36.3,37.1,36.5


In [ ]:
# transformation: données annuelles --> trimestrielles // Option 1: recopier les données annuelles 4 fois

# Création d'une copie du dataframe original
df_working_hours_quarterly_1 = df_working_hours_pivot_eurostat_an.copy()

# Répéter chaque année 4 fois
df_working_hours_quarterly_1 = df_working_hours_pivot_eurostat_an.loc[df_working_hours_pivot_eurostat_an.index.repeat(4)]

# Générer un nouvel index trimestriel sous le format 'YYYY-QX'
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
df_working_hours_quarterly_1['TIME_PERIOD'] = [f"{year}-{quarter}" for year in df_working_hours_pivot_eurostat_an.index for quarter in quarters]

# Remettre 'TIME_PERIOD' comme index
df_working_hours_quarterly_1 = df_working_hours_quarterly_1.set_index('TIME_PERIOD')

# Renommer les colonnes pour éviter la duplication des préfixes dans les noms des colonnes
df_working_hours_quarterly_1.columns = [col.replace("Working_Hours_Working_Hours", "Working_Hours") for col in df_working_hours_quarterly_1.columns]

# Vérificatino
df_working_hours_quarterly_1






,Working_Hours_Austria,Working_Hours_Belgium,Working_Hours_Czechia,Working_Hours_Denmark,Working_Hours_Estonia,Working_Hours_Finland,Working_Hours_France,Working_Hours_Germany,Working_Hours_Greece,Working_Hours_Hungary,...,Working_Hours_Netherlands,Working_Hours_Norway,Working_Hours_Poland,Working_Hours_Portugal,Working_Hours_Romania,Working_Hours_Slovenia,Working_Hours_Spain,Working_Hours_Sweden,Working_Hours_Switzerland,Working_Hours_United Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,40.0,37.8,NaN,34.3,NaN,36.9,39.7,38.7,42.3,NaN,...,30.5,34.4,NaN,41.6,NaN,NaN,38.3,34.0,NaN,36.9
1995-Q2,40.0,37.8,NaN,34.3,NaN,36.9,39.7,38.7,42.3,NaN,...,30.5,34.4,NaN,41.6,NaN,NaN,38.3,34.0,NaN,36.9
1995-Q3,40.0,37.8,NaN,34.3,NaN,36.9,39.7,38.7,42.3,NaN,...,30.5,34.4,NaN,41.6,NaN,NaN,38.3,34.0,NaN,36.9
1995-Q4,40.0,37.8,NaN,34.3,NaN,36.9,39.7,38.7,42.3,NaN,...,30.5,34.4,NaN,41.6,NaN,NaN,38.3,34.0,NaN,36.9
1996-Q1,39.8,37.8,NaN,34.0,NaN,37.6,39.6,38.7,41.9,41.2,...,32.3,34.9,NaN,40.8,NaN,42.2,38.0,34.4,37.0,36.7
1996-Q2,39.8,37.8,NaN,34.0,NaN,37.6,39.6,38.7,41.9,41.2,...,32.3,34.9,NaN,40.8,NaN,42.2,38.0,34.4,37.0,36.7
1996-Q3,39.8,37.8,NaN,34.0,NaN,37.6,39.6,38.7,41.9,41.2,...,32.3,34.9,NaN,40.8,NaN,42.2,38.0,34.4,37.0,36.7
1996-Q4,39.8,37.8,NaN,34.0,NaN,37.6,39.6,38.7,41.9,41.2,...,32.3,34.9,NaN,40.8,NaN,42.2,38.0,34.4,37.0,36.7
1997-Q1,40.2,37.9,43.4,34.4,41.3,39.4,39.2,38.6,41.8,42.0,...,32.3,35.7,42.2,40.0,40.3,40.5,38.3,36.3,37.1,36.5


In [47]:
#transformation: données annuelles--> trimestrielles // Option 2: Interpolation
# Création d'une copie du dataframe original
df_working_hours_quarterly_2 = df_working_hours_pivot_eurostat_an.copy()


# Idée pour l'interpolation: recopier la valeur annuelle pour le premier trimestre et
# ajouter des lignes "vides" entre le premier trimestre des deux années consécutives, puis faire l'interpolation
new_index = []
values = []

for year in df_working_hours_quarterly_2.index:
    # Ajouter Q1 pour chaque année (avec la valeur annuelle)
    new_index.append(f"{year}-Q1")
    values.append(df_working_hours_quarterly_2.loc[year].values)
    
    # Ajouter Q2, Q3 et Q4 (lignes vides pour l'instant)
    for quarter in ['Q2', 'Q3', 'Q4']:
        new_index.append(f"{year}-{quarter}")
        values.append([None] * len(df_working_hours_quarterly_2.columns))

# Création d'un nouveau DataFrame trimestriel avec les mêmes colonnes
df_working_hours_quarterly_2 = pd.DataFrame(values, index=new_index, columns=df_working_hours_quarterly_2.columns)

#On met le nom de "TIME_PERIOD"
df_working_hours_quarterly_2.index.name = 'TIME_PERIOD'

# Interpolation linéaire
df_working_hours_quarterly_2 = df_working_hours_quarterly_2.interpolate(method='linear')


# Renommer les colonnes pour éviter la duplication des préfixes dans les noms des colonnes
df_working_hours_quarterly_2.columns = [col.replace("Working_Hours_Working_Hours", "Working_Hours") for col in df_working_hours_quarterly_1.columns]

#Vérification
df_working_hours_quarterly_2


,Working_Hours_Austria,Working_Hours_Belgium,Working_Hours_Czechia,Working_Hours_Denmark,Working_Hours_Estonia,Working_Hours_Finland,Working_Hours_France,Working_Hours_Germany,Working_Hours_Greece,Working_Hours_Hungary,...,Working_Hours_Netherlands,Working_Hours_Norway,Working_Hours_Poland,Working_Hours_Portugal,Working_Hours_Romania,Working_Hours_Slovenia,Working_Hours_Spain,Working_Hours_Sweden,Working_Hours_Switzerland,Working_Hours_United Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,40.00,37.800,NaN,34.300,NaN,36.900,39.700,38.700,42.300,NaN,...,30.50,34.400,NaN,41.6,NaN,NaN,38.300,34.000,NaN,36.90
1995-Q2,39.95,37.800,NaN,34.225,NaN,37.075,39.675,38.700,42.200,NaN,...,30.95,34.525,NaN,41.4,NaN,NaN,38.225,34.100,NaN,36.85
1995-Q3,39.90,37.800,NaN,34.150,NaN,37.250,39.650,38.700,42.100,NaN,...,31.40,34.650,NaN,41.2,NaN,NaN,38.150,34.200,NaN,36.80
1995-Q4,39.85,37.800,NaN,34.075,NaN,37.425,39.625,38.700,42.000,NaN,...,31.85,34.775,NaN,41.0,NaN,NaN,38.075,34.300,NaN,36.75
1996-Q1,39.80,37.800,NaN,34.000,NaN,37.600,39.600,38.700,41.900,41.2,...,32.30,34.900,NaN,40.8,NaN,42.200,38.000,34.400,37.000,36.70
1996-Q2,39.90,37.825,NaN,34.100,NaN,38.050,39.500,38.675,41.875,41.4,...,32.30,35.100,NaN,40.6,NaN,41.775,38.075,34.875,37.025,36.65
1996-Q3,40.00,37.850,NaN,34.200,NaN,38.500,39.400,38.650,41.850,41.6,...,32.30,35.300,NaN,40.4,NaN,41.350,38.150,35.350,37.050,36.60
1996-Q4,40.10,37.875,NaN,34.300,NaN,38.950,39.300,38.625,41.825,41.8,...,32.30,35.500,NaN,40.2,NaN,40.925,38.225,35.825,37.075,36.55
1997-Q1,40.20,37.900,43.4,34.400,41.3,39.400,39.200,38.600,41.800,42.0,...,32.30,35.700,42.2,40.0,40.3,40.500,38.300,36.300,37.100,36.50


In [46]:
# Extraction des données Working Hours trimestriels (Données Eurostat à partir de 2008)
'''
Informations générales: données trimestrielles à partir de 2008
Online data code: lfsq_ewhan2
Unit of measure: Hours per year per person
'''

# Charger les données
df_working_hours_eurostat_1 = pd.read_csv("Données_extraites/Working_hours_eurostat.csv", encoding='utf-8')
#print(df_working_hours_eurostat_1.head())

# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_1 = df_working_hours_eurostat_1[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_working_hours_selected_eurostat_1.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_1 = df_working_hours_selected_eurostat_1.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure un préfixe "WorkingHours_" suivi du nom du pays
df_working_hours_pivot_eurostat_1.columns = [f'Working_Hours_{col}' for col in df_working_hours_pivot_eurostat_1.columns]

# Vérifier que les dates sont bien en index et les pays sont renommés
#print(df_working_hours_pivot_eurostat_1.head())
df_working_hours_pivot_eurostat_1

#On change le nom pour simplifier
df_working_hours_eurostat_1=df_working_hours_pivot_eurostat_1
df_working_hours_eurostat_1


,Working_Hours_Austria,Working_Hours_Belgium,Working_Hours_Bulgaria,Working_Hours_Croatia,Working_Hours_Cyprus,Working_Hours_Czechia,Working_Hours_Denmark,Working_Hours_Estonia,Working_Hours_Finland,Working_Hours_France,...,Working_Hours_Poland,Working_Hours_Portugal,Working_Hours_Romania,Working_Hours_Slovakia,Working_Hours_Slovenia,Working_Hours_Spain,Working_Hours_Sweden,Working_Hours_Switzerland,Working_Hours_Türkiye,Working_Hours_United Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
2008-Q1,37.6,36.7,40.7,39.6,38.6,41.3,34.3,39.3,35.8,36.7,...,39.8,38.1,39.4,39.6,39.3,37.7,34.8,NaN,48.9,35.2
2008-Q2,37.7,36.7,41.0,40.5,38.4,41.5,35.0,39.6,36.9,36.3,...,40.5,38.1,40.4,40.3,39.4,38.6,35.4,37.4,50.2,35.9
2008-Q3,38.7,37.2,41.8,40.7,40.7,42.3,36.2,40.1,38.2,37.5,...,41.5,39.7,40.5,39.6,40.3,39.0,36.7,NaN,51.4,36.4
2008-Q4,37.2,36.5,41.0,39.7,38.7,40.7,34.8,38.4,36.5,36.5,...,39.3,38.3,39.6,39.9,39.3,38.1,35.7,NaN,49.4,35.8
2009-Q1,36.7,36.9,39.8,39.3,37.8,40.9,34.8,37.6,35.9,36.9,...,39.5,38.6,39.0,39.2,39.4,37.6,35.1,NaN,47.2,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q3,34.1,34.4,39.3,39.1,39.5,37.0,34.0,37.0,35.3,36.0,...,40.2,38.4,39.8,37.2,38.5,36.9,36.0,36.6,45.2,NaN
2023-Q4,32.9,34.1,39.0,37.6,38.7,37.8,33.0,36.3,33.6,35.9,...,38.6,36.5,39.1,37.2,37.8,35.4,35.5,36.3,43.8,NaN
2024-Q1,34.0,35.0,39.3,38.1,37.7,37.7,32.6,35.9,34.2,36.2,...,39.2,37.1,39.2,38.0,37.7,36.1,35.3,35.7,42.9,NaN


In [ ]:
# Extraction des données Working Hours per year (Données Eurostat à partir de 1998-2008)
'''
Informations générales: données trimestrielles 1998--> 2008
Online data code: lfsq_ewhana
Source: https://ec.europa.eu/eurostat/databrowser/view/lfsq_ewhana__custom_15382513/default/table?lang=en
Unit of measure: Hours per year per person
'''

# Charger les données
df_working_hours_eurostat_2 = pd.read_csv("Données_extraites/Working_hours_eurostat_2.csv", encoding='utf-8')
#print(df_GDP.head())

# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_2 = df_working_hours_eurostat_2[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_GDP_selected.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_2 = df_working_hours_selected_eurostat_2.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure un préfixe (par exemple, "Working_Hours_")
df_working_hours_pivot_eurostat_2.columns = [f'Working_Hours_{col}' for col in df_working_hours_pivot_eurostat_2.columns]

# Supprimer les 4 dernières lignes (pour que l'année 2008 ne se répète pas 2 fois lors de la concaténation finale)
df_working_hours_pivot_eurostat_2 = df_working_hours_pivot_eurostat_2.iloc[:-4, :]

#On change le nom pour simplifier
df_working_hours_eurostat_2=df_working_hours_pivot_eurostat_2
df_working_hours_eurostat_2


,Working_Hours_Austria,Working_Hours_Belgium,Working_Hours_Bulgaria,Working_Hours_Croatia,Working_Hours_Cyprus,Working_Hours_Czechia,Working_Hours_Denmark,Working_Hours_Estonia,Working_Hours_Finland,Working_Hours_France,...,Working_Hours_Poland,Working_Hours_Portugal,Working_Hours_Romania,Working_Hours_Slovakia,Working_Hours_Slovenia,Working_Hours_Spain,Working_Hours_Sweden,Working_Hours_Switzerland,Working_Hours_Türkiye,Working_Hours_United Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1998-Q1,39.8,NaN,NaN,NaN,NaN,43.1,NaN,NaN,38.5,39.3,...,NaN,40.8,NaN,41.7,NaN,38.6,NaN,NaN,NaN,NaN
1998-Q2,NaN,38.5,NaN,NaN,NaN,43.5,33.9,41.1,37.8,NaN,...,41.9,39.9,39.9,42.0,40.7,38.5,34.5,37.1,NaN,36.5
1998-Q3,NaN,NaN,NaN,NaN,NaN,44.4,NaN,NaN,39.6,NaN,...,NaN,41.2,NaN,42.9,NaN,40.0,NaN,NaN,NaN,NaN
1998-Q4,NaN,NaN,NaN,NaN,NaN,44.2,NaN,NaN,38.9,NaN,...,NaN,39.1,NaN,42.7,NaN,38.2,NaN,NaN,NaN,NaN
1999-Q1,39.6,37.3,NaN,NaN,NaN,43.8,35.2,NaN,38.4,38.9,...,39.8,40.1,38.2,41.9,41.0,38.6,NaN,NaN,NaN,NaN
1999-Q2,38.8,36.8,NaN,NaN,39.7,43.9,35.5,41.1,37.5,NaN,...,NaN,40.0,40.0,42.1,41.4,39.5,36.4,37.3,NaN,36.3
1999-Q3,40.2,37.7,NaN,NaN,NaN,44.0,36.7,NaN,39.2,NaN,...,NaN,40.7,40.6,42.6,42.1,40.0,NaN,NaN,NaN,37.2
1999-Q4,39.5,36.5,NaN,NaN,NaN,43.4,35.4,NaN,37.0,NaN,...,NaN,39.0,38.8,42.2,41.9,37.6,NaN,NaN,NaN,37.1
2000-Q1,39.5,37.6,40.2,NaN,NaN,43.6,36.1,40.4,37.6,38.5,...,40.1,40.1,38.0,42.1,41.1,39.4,NaN,NaN,NaN,35.6


Ainsi nous obtenons les données sur les Working Hours

Nom du data_frame: df_working_hours

In [50]:
# Concaténation verticale pour fusionner les trois jeux de données 
# Remarque: pour la période 1995-1997, nous avons pris la méthode par l'interpolation linéaire

df_working_hours = pd.concat(
    [df_working_hours_quarterly_2, df_working_hours_pivot_eurostat_2, df_working_hours_pivot_eurostat_1], 
    axis=0
)

df_working_hours

,Working_Hours_Austria,Working_Hours_Belgium,Working_Hours_Czechia,Working_Hours_Denmark,Working_Hours_Estonia,Working_Hours_Finland,Working_Hours_France,Working_Hours_Germany,Working_Hours_Greece,Working_Hours_Hungary,...,Working_Hours_Switzerland,Working_Hours_United Kingdom,Working_Hours_Bulgaria,Working_Hours_Croatia,Working_Hours_Cyprus,Working_Hours_Latvia,Working_Hours_Lithuania,Working_Hours_Malta,Working_Hours_Slovakia,Working_Hours_Türkiye
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,40.00,37.8,NaN,34.300,NaN,36.900,39.700,38.7,42.3,NaN,...,NaN,36.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-Q2,39.95,37.8,NaN,34.225,NaN,37.075,39.675,38.7,42.2,NaN,...,NaN,36.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-Q3,39.90,37.8,NaN,34.150,NaN,37.250,39.650,38.7,42.1,NaN,...,NaN,36.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-Q4,39.85,37.8,NaN,34.075,NaN,37.425,39.625,38.7,42.0,NaN,...,NaN,36.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-Q1,39.80,37.8,NaN,34.000,NaN,37.600,39.600,38.7,41.9,41.2,...,37.0,36.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q3,34.10,34.4,37.0,34.000,37.0,35.300,36.000,34.4,40.7,38.8,...,36.6,NaN,39.3,39.1,39.5,38.9,38.7,37.2,37.2,45.2
2023-Q4,32.90,34.1,37.8,33.000,36.3,33.600,35.900,33.5,39.5,36.4,...,36.3,NaN,39.0,37.6,38.7,38.4,37.6,36.9,37.2,43.8
2024-Q1,34.00,35.0,37.7,32.600,35.9,34.200,36.200,33.7,38.9,37.0,...,35.7,NaN,39.3,38.1,37.7,38.3,37.8,36.6,38.0,42.9


Données des taux d'intérêts

Taux court terme : "Les données se réfèrent aux bons acceptés par la banque/Certificats de dépôt négociables-3 mois."
Nom du dataframe: df_ST_IR


Taux long terme : "Les données se réfèrent au taux de rendement estimé en clôture des obligations du trésor à 10 ans."
Nom du dataframe: df_LT_IR

In [66]:
# Pour le long terme (une possibilité): Long term interest rates
'''
Source: https://data-explorer.oecd.org/vis?lc=en&pg=0&tm=LONG-TERM%20INTEREST
Unit of measure: Percent per annum
'''

# Charger les données
df_LT_IR = pd.read_csv('Données_extraites/Long_term_IR_OCDE.csv', encoding='utf-8')

# Choix des colonnes rélévantes
df_LT_IR_selected = df_LT_IR[['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]

# Obtention des données de panel en format séries temporelles
df_LT_IR_pivot = df_LT_IR_selected.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure le préfixe LT_IR_ (=Long Term Interest Rate)
df_LT_IR_pivot.columns = [f'LT_IR_{col}' for col in df_LT_IR_pivot.columns]
df_LT_IR_pivot = df_LT_IR_pivot.rename(columns={'LT_IR_Slovak Republic': 'LT_IR_Slovakia'})

# On simplifie le nom, et on vérifie
df_LT_IR=df_LT_IR_pivot
df_LT_IR


,LT_IR_Austria,LT_IR_Belgium,LT_IR_Bulgaria,LT_IR_Croatia,LT_IR_Czechia,LT_IR_Denmark,LT_IR_Estonia,LT_IR_Finland,LT_IR_France,LT_IR_Germany,...,LT_IR_Norway,LT_IR_Poland,LT_IR_Portugal,LT_IR_Romania,LT_IR_Slovakia,LT_IR_Slovenia,LT_IR_Spain,LT_IR_Sweden,LT_IR_Switzerland,LT_IR_United Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,7.627367,8.293333,NaN,NaN,NaN,8.960000,NaN,10.213330,8.088200,7.432434,...,8.046667,NaN,11.823330,NaN,NaN,NaN,11.906670,10.963330,5.275333,8.661467
1995-Q2,7.184033,7.590000,NaN,NaN,NaN,8.443666,NaN,8.983334,7.592500,6.911238,...,7.590000,NaN,11.990000,NaN,NaN,NaN,11.680000,10.913330,4.947333,8.242567
1995-Q3,6.997967,7.186666,NaN,NaN,NaN,8.087667,NaN,8.353333,7.377333,6.731236,...,7.353333,NaN,11.383330,NaN,NaN,NaN,11.033330,10.136670,4.714667,8.108933
1995-Q4,6.730933,6.853333,NaN,NaN,NaN,7.583000,NaN,7.616667,7.080767,6.327264,...,6.706666,NaN,10.663330,NaN,NaN,NaN,10.460000,8.963333,4.175667,7.788167
1996-Q1,6.396633,6.640000,NaN,NaN,NaN,7.349000,NaN,7.470000,6.577667,6.178932,...,6.676667,NaN,9.450000,NaN,NaN,NaN,9.683333,8.580000,4.400333,7.753200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q4,3.125533,3.150000,4.023334,3.833333,4.371921,2.787333,3.656667,3.116667,3.096667,2.508883,...,3.640197,5.530000,3.246667,6.583333,3.766608,3.486667,3.583333,2.662297,0.841333,4.234600
2024-Q1,2.835333,2.890000,3.933333,3.293333,3.821404,2.423667,3.430000,2.844653,2.803333,2.285317,...,3.601987,5.333333,3.016667,6.053333,3.470253,3.160000,3.216667,2.352227,0.765667,4.029300
2024-Q2,3.013533,3.063333,3.930000,3.376667,4.190742,2.506667,3.706667,2.999653,3.050000,2.483253,...,3.652627,5.690000,3.160000,6.176667,3.687122,3.256667,3.313333,2.382347,0.722333,4.201500


In [67]:
# Pour le court terme (une possibilité): short term interest rates

'''
Informations générales 
source: https://data-explorer.oecd.org/vis?lc=en&pg=0&tm=LONG-TERM%20INTEREST&snb=21&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_STES%40DF_FINMARK&df[ag]=OECD.SDD.STES&df[vs]=4.0&dq=GBR%2BROU%2BHRV%2BBGR%2BESP%2BTUR%2BCHE%2BSWE%2BSVN%2BSVK%2BPRT%2BPOL%2BNOR%2BNLD%2BLUX%2BLTU%2BLVA%2BITA%2BIRL%2BISL%2BHUN%2BGRC%2BDEU%2BFRA%2BFIN%2BEST%2BDNK%2BCZE%2BBEL%2BAUT%2BAUS.Q.IRLT.PA.....&to[TIME_PERIOD]=false&pd=1995-Q1%2C
Unit of measure: Percent per annum
'''

df_ST_IR=pd.read_csv('Données_extraites/Short_term_IR_OCDE.csv', encoding='utf-8')

# Choix des colonnes rélévantes
df_ST_IR_selected = df_ST_IR[['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]

# Obtention des données de panel en format séries temporelles
df_ST_IR_pivot = df_ST_IR_selected.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure le préfixe ST_IR_ (=Short Term Interest Rate)
df_ST_IR_pivot.columns = [f'ST_IR_{col}' for col in df_ST_IR_pivot.columns]
df_ST_IR_pivot = df_ST_IR_pivot.rename(columns={'ST_IR_Slovak Republic': 'ST_IR_Slovakia'})


# On simplifie le nom, et on vérifie
df_ST_IR=df_ST_IR_pivot
df_ST_IR


,ST_IR_Austria,ST_IR_Belgium,ST_IR_Bulgaria,ST_IR_Czechia,ST_IR_Denmark,ST_IR_Estonia,ST_IR_Finland,ST_IR_France,ST_IR_Germany,ST_IR_Greece,...,ST_IR_Norway,ST_IR_Poland,ST_IR_Portugal,ST_IR_Romania,ST_IR_Slovakia,ST_IR_Slovenia,ST_IR_Spain,ST_IR_Sweden,ST_IR_Switzerland,ST_IR_United Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,5.043334,5.706666,NaN,10.858710,6.495430,NaN,5.966667,6.600767,5.110000,18.116840,...,5.536667,29.710000,10.527000,NaN,NaN,NaN,9.032666,8.196667,NaN,6.722410
1995-Q2,4.680000,5.050000,NaN,10.501130,6.854634,NaN,5.953333,7.482500,4.600000,16.467290,...,5.606667,28.856670,10.200330,NaN,NaN,NaN,9.454333,8.903334,NaN,6.742444
1995-Q3,4.373333,4.416667,NaN,11.310300,6.110099,NaN,6.053333,6.122233,4.403333,15.321020,...,5.440000,26.526670,9.337000,NaN,6.983333,NaN,9.543667,9.110000,NaN,6.849310
1995-Q4,4.176667,3.960000,NaN,11.114370,5.350968,NaN,5.026667,6.107234,4.013333,15.711430,...,5.330000,25.666670,9.082334,47.636670,8.156667,NaN,9.390000,8.773334,NaN,6.696187
1996-Q1,3.476667,3.350000,NaN,10.860570,4.451957,8.226666,4.163333,4.460033,3.440000,14.225530,...,5.220000,22.756670,8.166667,58.903330,8.836667,NaN,8.680000,7.643333,NaN,6.277534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q4,3.957456,3.957456,1.419333,7.025323,3.941422,3.957456,3.957456,3.957456,3.957456,3.957456,...,4.713333,5.766667,3.957456,6.306667,3.957456,3.957456,3.957456,4.028850,1.718106,5.326667
2024-Q1,3.923615,3.923615,1.665700,6.224104,3.882079,3.923615,3.923615,3.923615,3.923615,3.923615,...,4.716667,5.863333,3.923615,6.106667,3.923615,3.923615,3.923615,4.022277,1.628413,5.203333
2024-Q2,3.808172,3.808172,1.787933,5.248055,3.734747,3.808172,3.808172,3.808172,3.808172,3.808172,...,4.720000,5.850000,3.808172,6.043334,3.808172,3.808172,3.808172,3.807240,1.475667,5.166667


In [68]:
print(df_GDP.columns)
print(df_working_hours.columns)
print(df_LT_IR.columns)
print(df_ST_IR.columns)




Index(['PIB_Albania', 'PIB_Austria', 'PIB_Belgium',
       'PIB_Bosnia and Herzegovina', 'PIB_Bulgaria', 'PIB_Croatia',
       'PIB_Cyprus', 'PIB_Czechia', 'PIB_Denmark', 'PIB_Estonia',
       'PIB_Finland', 'PIB_France', 'PIB_Germany', 'PIB_Greece', 'PIB_Hungary',
       'PIB_Iceland', 'PIB_Ireland', 'PIB_Italy', 'PIB_Kosovo*', 'PIB_Latvia',
       'PIB_Lithuania', 'PIB_Luxembourg', 'PIB_Malta', 'PIB_Montenegro',
       'PIB_Netherlands', 'PIB_North Macedonia', 'PIB_Norway', 'PIB_Poland',
       'PIB_Portugal', 'PIB_Romania', 'PIB_Serbia', 'PIB_Slovakia',
       'PIB_Slovenia', 'PIB_Spain', 'PIB_Sweden', 'PIB_Switzerland',
       'PIB_Türkiye', 'PIB_United Kingdom'],
      dtype='object')
Index(['Working_Hours_Austria', 'Working_Hours_Belgium',
       'Working_Hours_Czechia', 'Working_Hours_Denmark',
       'Working_Hours_Estonia', 'Working_Hours_Finland',
       'Working_Hours_France', 'Working_Hours_Germany', 'Working_Hours_Greece',
       'Working_Hours_Hungary', 'Working_Hours_Ice

Import du CPI

In [ ]:
# données téléchargées depuis le site de eurostat
# Décompresser le fichier qui était en .gz
df_cpi = pd.read_csv("estat_prc_hicp_midx_filtered_en.csv.gz", compression="gzip")
df_cpi




                   DATAFLOW        LAST UPDATE     freq             unit  \
0  ESTAT:PRC_HICP_MIDX(1.0)  03/02/25 11:00:00  Monthly  Index, 2015=100   
1  ESTAT:PRC_HICP_MIDX(1.0)  03/02/25 11:00:00  Monthly  Index, 2015=100   
2  ESTAT:PRC_HICP_MIDX(1.0)  03/02/25 11:00:00  Monthly  Index, 2015=100   
3  ESTAT:PRC_HICP_MIDX(1.0)  03/02/25 11:00:00  Monthly  Index, 2015=100   
4  ESTAT:PRC_HICP_MIDX(1.0)  03/02/25 11:00:00  Monthly  Index, 2015=100   

           coicop      geo TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0  All-items HICP  Albania     2015-12     100.00        d          NaN  
1  All-items HICP  Albania     2016-01     101.00        d          NaN  
2  All-items HICP  Albania     2016-02     102.28        d          NaN  
3  All-items HICP  Albania     2016-03     102.80        d          NaN  
4  All-items HICP  Albania     2016-04     102.31        d          NaN  


In [64]:
# données téléchargées depuis le site de eurostat

# décompresser le fichier qui était en .gz
df_cpi = pd.read_csv("estat_prc_hicp_midx_filtered_en.csv.gz", compression="gzip")


print(df_cpi['geo'].unique())

'''
# Sélectionner les variables utiles
df_cpi = df_cpi[['coicop','geo', 'TIME_PERIOD', 'OBS_VALUE']]
print(df_cpi.head())
print(df_cpi['coicop'].unique())

# mettre les deux variables de cpi différentes dans deux colonnes (car actuellement elles sont en ligne : voir la variable coicop))
df_cpi_pivot = df_cpi.pivot(index=['geo', 'TIME_PERIOD'], columns='coicop', values='OBS_VALUE')
print(df_cpi_pivot.head())
df_cpi_pivot = df_cpi_pivot.reset_index()
df_cpi_pivot
'''

['Albania' 'Austria' 'Belgium' 'Bulgaria' 'Switzerland' 'Cyprus' 'Czechia'
 'Germany' 'Denmark' 'Estonia' 'Greece' 'Spain' 'Finland' 'France'
 'Croatia' 'Hungary' 'Ireland' 'Iceland' 'Italy' 'Lithuania' 'Luxembourg'
 'Latvia' 'Montenegro' 'North Macedonia' 'Malta' 'Netherlands' 'Norway'
 'Poland' 'Portugal' 'Romania' 'Serbia' 'Sweden' 'Slovenia' 'Slovakia'
 'Türkiye' 'United Kingdom' 'United States' 'Kosovo*']


"\n# Sélectionner les variables utiles\ndf_cpi = df_cpi[['coicop','geo', 'TIME_PERIOD', 'OBS_VALUE']]\nprint(df_cpi.head())\nprint(df_cpi['coicop'].unique())\n\n# mettre les deux variables de cpi différentes dans deux colonnes (car actuellement elles sont en ligne : voir la variable coicop))\ndf_cpi_pivot = df_cpi.pivot(index=['geo', 'TIME_PERIOD'], columns='coicop', values='OBS_VALUE')\nprint(df_cpi_pivot.head())\ndf_cpi_pivot = df_cpi_pivot.reset_index()\ndf_cpi_pivot\n"

In [51]:
#tables en format long à merger : 
set1 = set(df_working_hours_long_eurostat['geo'].unique())
set2 = set(df_cpi_pivot['geo'].unique())
set3 = set(df_ST_IR['Reference area'].unique())
set4 = set(df_LT_IR['Reference area'].unique())
set5 = set(df_GDP_selected['geo'].unique())

pays_communs = set1 & set2 & set3 & set4 & set5  # Intersection de tous les ensembles
print("Pays communs :", pays_communs)
print("Nombre de pays communs :", len(pays_communs))



Pays communs : {'Denmark', 'Czechia', 'Italy', 'Norway', 'France', 'Romania', 'Germany', 'Poland', 'Bulgaria', 'Belgium', 'Estonia', 'Sweden', 'Switzerland', 'Greece', 'Spain', 'Finland', 'Portugal', 'Iceland', 'Slovenia', 'Austria', 'United Kingdom', 'Latvia', 'Netherlands', 'Ireland', 'Lithuania', 'Hungary', 'Luxembourg'}
Nombre de pays communs : 27


In [124]:
tous_pays = set1 | set2 | set3 | set4 | set5  # Union de tous les ensembles
print("Tous les pays :", tous_pays)

for i, (name, dataset) in enumerate([
    ("df_working_hours_long_eurostat", set1),
    ("df_cpi_pivot", set2),
    ("df_ST_IR", set3),
    ("df_LT_IR", set4),
    ("df_GDP_selected", set5)
]):
    pays_absents = tous_pays - dataset  # Pays qui ne sont pas dans cet ensemble
    print(f"Pays absents dans {name} :", pays_absents)

#suppression des US dans df_cpi_pivot :
df_cpi_pivot = df_cpi_pivot[~df_cpi_pivot['geo'].isin(['United States'])]

Tous les pays : {'Denmark', 'Kosovo*', 'Bosnia and Herzegovina', 'France', 'Poland', 'Bulgaria', 'Türkiye', 'Estonia', 'Sweden', 'Serbia', 'Switzerland', 'Greece', 'Finland', 'Iceland', 'North Macedonia', 'Austria', 'United Kingdom', 'Albania', 'Lithuania', 'Hungary', 'Luxembourg', 'Czechia', 'Italy', 'Norway', 'Slovakia', 'Romania', 'Germany', 'Croatia', 'Belgium', 'Cyprus', 'Slovak Republic', 'Spain', 'Montenegro', 'United States', 'Malta', 'Portugal', 'Slovenia', 'Latvia', 'Netherlands', 'Ireland'}
Pays absents dans df_working_hours_long_eurostat : {'Kosovo*', 'Montenegro', 'United States', 'Bosnia and Herzegovina', 'Serbia', 'Albania', 'North Macedonia', 'Slovak Republic'}
Pays absents dans df_cpi_pivot : {'Slovak Republic', 'Bosnia and Herzegovina'}
Pays absents dans df_ST_IR : {'Croatia', 'Türkiye', 'Kosovo*', 'Cyprus', 'Montenegro', 'United States', 'Bosnia and Herzegovina', 'Malta', 'Serbia', 'North Macedonia', 'Albania', 'Slovakia'}
Pays absents dans df_LT_IR : {'Türkiye', 'Ko

In [125]:
# problème: les données sont mensuelles et elles n'existent pas en trimestriel sur eurostat : on va faire utiliser la moyenne pour chaque trimestre

# Convertir 'TIME_PERIOD' en datetime
df_cpi=df_cpi_pivot
df_cpi['TIME_PERIOD'] = pd.to_datetime(df_cpi['TIME_PERIOD'], format='%Y-%m')

# Ajouter une colonne 'quTIME_PERIODarter' en fonction de l'année et du mois
df_cpi['TIME_PERIOD'] = df_cpi['TIME_PERIOD'].dt.to_period('Q')

# Regrouper par pays et trimestre, puis prendre la moyenne
df_cpi_trimestriel = df_cpi.groupby(['geo', 'TIME_PERIOD'])[['All-items HICP','Overall index excluding energy, food, alcohol and tobacco']].mean().reset_index()
df_cpi_trimestriel['TIME_PERIOD'] = df_cpi_trimestriel['TIME_PERIOD'].astype(str).str.replace('Q', '-Q')

df_cpi_trimestriel


/tmp/ipykernel_5469/3327649979.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cpi['TIME_PERIOD'] = pd.to_datetime(df_cpi['TIME_PERIOD'], format='%Y-%m')
/tmp/ipykernel_5469/3327649979.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cpi['TIME_PERIOD'] = df_cpi['TIME_PERIOD'].dt.to_period('Q')


coicop,geo,TIME_PERIOD,All-items HICP,"Overall index excluding energy, food, alcohol and tobacco"
0,Albania,2015-Q4,100.000000,100.000000
1,Albania,2016-Q1,102.026667,99.826667
2,Albania,2016-Q2,101.143333,99.950000
3,Albania,2016-Q3,101.093333,100.383333
4,Albania,2016-Q4,101.770000,100.710000
...,...,...,...,...
3938,United Kingdom,2019-Q4,108.433333,108.233333
3939,United Kingdom,2020-Q1,108.466667,108.200000
3940,United Kingdom,2020-Q2,108.533333,108.766667
3941,United Kingdom,2020-Q3,108.933333,109.200000


In [126]:
# renommer les variables de tous les df pour qu'elles soient uniformes 
df_working_hours_long_eurostat = df_working_hours_long_eurostat.rename(columns={'geo': 'country', 'OBS_VALUE': 'working_hours'})
df_cpi_trimestriel = df_cpi_trimestriel.rename(columns={'geo': 'country'})
df_ST_IR = df_ST_IR.rename(columns={'Reference area': 'country', 'OBS_VALUE': 'short term'})
df_LT_IR = df_LT_IR.rename(columns={'Reference area': 'country', 'OBS_VALUE': 'long term'})
df_GDP_selected = df_GDP_selected.rename(columns={'geo': 'country', 'OBS_VALUE': 'gdp'})




In [127]:
# Maintenant on merge

from functools import reduce


# Liste de DataFrames à merger
dfs = [df_GDP_selected, df_ST_IR, df_LT_IR, df_working_hours_long_eurostat,df_cpi_trimestriel ]  

# Fusionner tous les DataFrames de la liste
df_merged = reduce(lambda left, right: left.merge(right, on=['country', 'TIME_PERIOD'], how='outer'), dfs)
df_merged



,country,TIME_PERIOD,gdp,short term,long term,working_hours,All-items HICP,"Overall index excluding energy, food, alcohol and tobacco"
0,Albania,2008-Q1,1924.3,NaN,NaN,NaN,NaN,NaN
1,Albania,2008-Q2,2281.3,NaN,NaN,NaN,NaN,NaN
2,Albania,2008-Q3,2257.7,NaN,NaN,NaN,NaN,NaN
3,Albania,2008-Q4,2338.9,NaN,NaN,NaN,NaN,NaN
4,Albania,2009-Q1,1992.2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4490,United Kingdom,2023-Q4,NaN,5.326667,4.234600,NaN,NaN,NaN
4491,United Kingdom,2024-Q1,NaN,5.203333,4.029300,NaN,NaN,NaN
4492,United Kingdom,2024-Q2,NaN,5.166667,4.201500,NaN,NaN,NaN
4493,United Kingdom,2024-Q3,NaN,4.946667,3.996100,NaN,NaN,NaN
